In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['AZURE_OPENAI_KEY']=os.getenv("AZURE_OPENAI_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [2]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"), 
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
    api_key = os.getenv('AZURE_OPENAI_KEY'),  
    temperature=0.9  
)



In [3]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
loader=WebBaseLoader("https://jamesclear.com/atomic-habits-summary")
loader

In [5]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://jamesclear.com/atomic-habits-summary', 'title': 'Atomic Habits Summary by James Clear', 'description': "Read this Atomic Habits summary to glean 3 key lessons from the book and learn James Clear's simple rules for better habits.", 'language': 'en-US'}, page_content='\n\n\n\n\n\n\n\n\n\n\nAtomic Habits Summary by James Clear\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBooks\nArticles\nNewsletter\nApp\nSpeaking\n\n\n\n\n\nMenu\nClose\n\n\nAbout\n\n\n\nBooks\nArticles\nNewsletter\nApp\nSpeaking\nAbout\n\n\n\n\n\n\n\n\n\n\n\n\nAtomic Habits Summary\nAtomic Habits Summary in Three Sentences\nAtomic Habits by James Clear is a comprehensive, practical guide on how to change your habits and get 1% better every day. Using a framework called the Four Laws of Behavior Change, Atomic Habits teaches readers a simple set of rules for creating good habi

In [6]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [7]:
documents

[Document(metadata={'source': 'https://jamesclear.com/atomic-habits-summary', 'title': 'Atomic Habits Summary by James Clear', 'description': "Read this Atomic Habits summary to glean 3 key lessons from the book and learn James Clear's simple rules for better habits.", 'language': 'en-US'}, page_content='Atomic Habits Summary by James Clear\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBooks\nArticles\nNewsletter\nApp\nSpeaking\n\n\n\n\n\nMenu\nClose\n\n\nAbout\n\n\n\nBooks\nArticles\nNewsletter\nApp\nSpeaking\nAbout\n\n\n\n\n\n\n\n\n\n\n\n\nAtomic Habits Summary\nAtomic Habits Summary in Three Sentences\nAtomic Habits by James Clear is a comprehensive, practical guide on how to change your habits and get 1% better every day. Using a framework called the Four Laws of Behavior Change, Atomic Habits teaches readers a simple set of rules for creating good habits and breaking bad on

In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS


# Initialize OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text") 

# Create the FAISS vector database
vectorstore = FAISS.from_documents(documents, embeddings) 


In [10]:
vectorstore

In [11]:
## Query From a vector db
query="good habits take time"
result=vectorstore.similarity_search(query)
result[0].page_content

'Atomic Habits\n\nBuy the book:\nPrint Kindle Audible \nOr, learn more about Atomic Habits.\n\n\n\n\n\n\n30 Days to Better Habits: A simple step-by-step guide for forming habits that stick\n\n\nTake the guesswork out of habit-building. 11 email lessons walk you through the first 30 days of a habit step-by-step, so you know exactly what to do.\nGet the tools and strategies you need to take action. The course includes a 20-page PDF workbook (including templates and cheatsheets), plus new examples and applications that you can’t find in Atomic Habits.\xa0\nLearn a framework that works for any habit. You can use this course to build any good habit – from getting fit, to saving for an early retirement, to daily meditation.\n\n\n\n\n\n\nEnroll in the free email course and get your first lesson today\n\nYou will get one short email every three days for a month. You can unsubscribe any time.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                Atomic Habits'

In [12]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| AzureChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000016C1EC71A00>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000016C1F15A990>, root_client=<openai.lib.azure.AzureOpenAI object at 0x0000016C1E935D30>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x0000016C1EC71A60>, temperature=0.9, model_kwargs={}, openai_api_key=SecretStr('**********'), disabl

In [13]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"what are good habits ",
    "context":[Document(page_content="If you’re having trouble changing your habits, the problem isn’t you. The problem is your system. Bad habits repeat themselves again and again not because you don’t want to change, but because you have the wrong system for change.")]
})

'The problem with changing habits is not due to a lack of desire to change, but rather due to having the wrong system for change.'

In [ ]:
### Input--->Retriever--->vectorstore
vectorstore

In [15]:
retriever=vectorstore.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [16]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016C3EC75D60>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [17]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"what ois the main learnign from this book"})
response['answer']

'To build better habits, follow the framework outlined in the Four Laws of Behavior Change: cue, craving, response, and reward. These four steps help clarify what a habit is, how it operates, and how it can be improved. Focus on systems rather than single accomplishments to ensure long-term progress through continuous refinement and commitment to the process. Small, consistent improvements over time can yield remarkable results.'

In [18]:
response

{'input': 'what ois the main learnign from this book',
 'context': [Document(id='19f087bd-e802-4934-97b8-ecd3d5780727', metadata={'source': 'https://jamesclear.com/atomic-habits-summary', 'title': 'Atomic Habits Summary by James Clear', 'description': "Read this Atomic Habits summary to glean 3 key lessons from the book and learn James Clear's simple rules for better habits.", 'language': 'en-US'}, page_content='Your identity emerges out of your habits. Every action is a vote for the type of person you wish to become.\nHow to build better habits in 4 simple steps\nThis section of the Atomic Habits summary presents key points from the actionable strategies in the book, organized around a framework called the Four Laws of Behavior Change.\xa0\xa0\xa0\nThe process of building a habit can be divided into four simple steps: cue, craving, response, and reward.\xa0\nBreaking it down into these fundamental parts can help us understand what a habit is, how it works, and how to improve it.'),
  

In [19]:
response['context']

[Document(id='19f087bd-e802-4934-97b8-ecd3d5780727', metadata={'source': 'https://jamesclear.com/atomic-habits-summary', 'title': 'Atomic Habits Summary by James Clear', 'description': "Read this Atomic Habits summary to glean 3 key lessons from the book and learn James Clear's simple rules for better habits.", 'language': 'en-US'}, page_content='Your identity emerges out of your habits. Every action is a vote for the type of person you wish to become.\nHow to build better habits in 4 simple steps\nThis section of the Atomic Habits summary presents key points from the actionable strategies in the book, organized around a framework called the Four Laws of Behavior Change.\xa0\xa0\xa0\nThe process of building a habit can be divided into four simple steps: cue, craving, response, and reward.\xa0\nBreaking it down into these fundamental parts can help us understand what a habit is, how it works, and how to improve it.'),
 Document(id='5e416b1b-746b-420e-bc33-0ac5171e1e7b', metadata={'sourc